In [1]:
import os
import shutil
from pathlib import Path

# Define source and destination directories
source_dir = Path('CAESAR/output_dataset')
destination_dir = Path('data/segmentation_dataset')

# Create subdirectories in the destination
for i in range(10):
    # Each subfolder gets an 'images' and 'masks' subdirectory
    os.makedirs(destination_dir / f'subfolder_{i+1}' / 'images', exist_ok=True)
    os.makedirs(destination_dir / f'subfolder_{i+1}' / 'masks', exist_ok=True)

# Function to distribute matching images and masks
def distribute_files():
    # Get all image files in the source images subdirectory
    image_files = os.listdir(source_dir / 'images')
    num_files = len(image_files)
    
    # Iterate through image files
    for index, image_name in enumerate(image_files):
        mask_name = image_name  # Assuming mask names match image names exactly

        # Determine which subfolder to use based on file index
        subfolder_index = index % 10  # This will distribute files evenly across 10 subfolders

        # Define the target subdirectories for images and masks
        target_image_subdir = destination_dir / f'subfolder_{subfolder_index+1}' / 'images'
        target_mask_subdir = destination_dir / f'subfolder_{subfolder_index+1}' / 'masks'

        # Move image
        shutil.move(str(source_dir / 'images' / image_name), str(target_image_subdir / image_name))
        # Move mask
        shutil.move(str(source_dir / 'masks' / mask_name), str(target_mask_subdir / mask_name))

distribute_files()

print("Files have been successfully moved and distributed, ensuring images and masks match in each subfolder.")

Files have been successfully moved and distributed, ensuring images and masks match in each subfolder.
